**Dual Stage Attention SpyNet (DAS):** The DAS Model integrates the Spatial Pyramid Network with CBAM to enhance feature extraction for anomaly localization in video surveillance using optical flow. It is trained with DSAPM-generated future frames, and anomalies are detected based on flow scores computed from End-Point Error (EPE).

In [ ]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, Add, BatchNormalization, Activation, Multiply, GlobalAveragePooling2D, Reshape
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split

# CBAM Attention Module
def cbam_block(input_tensor, reduction_ratio=16):
    """ CBAM (Convolutional Block Attention Module) """
    channel = GlobalAveragePooling2D()(input_tensor)
    channel = Reshape((1, 1, input_tensor.shape[-1]))(channel)
    channel = Conv2D(input_tensor.shape[-1] // reduction_ratio, (1, 1), activation='relu', padding='same')(channel)
    channel = Conv2D(input_tensor.shape[-1], (1, 1), activation='sigmoid', padding='same')(channel)
    channel_att = Multiply()([input_tensor, channel])

    spatial = Conv2D(1, (7, 7), activation='sigmoid', padding='same')(channel_att)
    spatial_att = Multiply()([channel_att, spatial])
    return spatial_att

# SpyNet Block
def spynet_block(input_tensor):
    x = Conv2D(64, (7, 7), padding="same", activation="relu")(input_tensor)
    x = Conv2D(64, (5, 5), padding="same", activation="relu")(x)
    x = cbam_block(x)
    x = Conv2D(32, (3, 3), padding="same", activation="relu")(x)
    x = Conv2D(2, (3, 3), padding="same")(x)  # Output Optical Flow (Vx, Vy)
    return x

# DAS Model (SpyNet + CBAM)
def build_das_model():
    input_layer = Input(shape=(128, 128, 2))
    output_layer = spynet_block(input_layer)
    model = Model(inputs=input_layer, outputs=output_layer, name="DAS_SpyNet_CBAM")
    return model

das_model = build_das_model()
das_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="mse")

def load_data(im_dir):
    all_images = []
    list_of_files = sorted(os.listdir(im_dir))

    for im_folder in list_of_files:
        list_of_img_files = sorted(os.listdir(os.path.join(im_dir, im_folder)))
        for image_file in list_of_img_files:
            image_path = os.path.join(im_dir, im_folder, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (128, 128))
            all_images.append([image])

    all_images = np.array(all_images).reshape(-1, 20, 128, 128, 1)
    return all_images

# Load dataset
train_data = load_data("/content/drive/MyDrive/Train_Data")
test_data = load_data("/content/drive/MyDrive/Test_Data")

# Load DSAPM model
dsapm_model = load_model("models/dsapm.h5")

# Prepare training data
X_train, Y_train = [], []
for sequence in train_data:
    for i in range(18):
        Z_n = sequence[i, ..., 0]
        Z_n1 = sequence[i+1, ..., 0]
        X_train.append(np.stack([Z_n, Z_n1], axis=-1))
        V_gt = das_model.predict(np.expand_dims(np.stack([Z_n, Z_n1], axis=-1), axis=0))
        Y_train.append(V_gt[0])

X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Train-test split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Train DAS model
das_model.fit(X_train, Y_train, batch_size=16, epochs=20, validation_data=(X_val, Y_val))
das_model.save("models/das_spynet_cbam_trained.h5")

# Compute End-Point Error (EPE)
def compute_epe(predicted_flow, gt_flow):
    return np.sqrt(np.sum((predicted_flow - gt_flow) ** 2, axis=-1)).mean()

# Compute flow threshold
normal_epe_values = []
for sequence in train_data:
    for i in range(17):
        Z_n = sequence[i, ..., 0]
        Z_n1 = sequence[i+1, ..., 0]
        V_gt = das_model.predict(np.expand_dims(np.stack([Z_n, Z_n1], axis=-1), axis=0))
        V_L = das_model.predict(np.expand_dims(np.stack([Z_n, sequence[i+1, ..., 0]], axis=-1), axis=0))
        normal_epe_values.append(compute_epe(V_L, V_gt))

flow_scores = (np.array(normal_epe_values) - np.min(normal_epe_values)) / (np.max(normal_epe_values) - np.min(normal_epe_values))
Fl_th = np.max(flow_scores)  # Threshold

# Test phase
for sequence in test_data:
    for i in range(18):
        Z_n = sequence[i, ..., 0]
        Z_n1 = sequence[i+1, ..., 0]
        V_gt = das_model.predict(np.expand_dims(np.stack([Z_n, Z_n1], axis=-1), axis=0))
        V_L = das_model.predict(np.expand_dims(np.stack([Z_n, sequence[i+1, ..., 0]], axis=-1), axis=0))
        epe = compute_epe(V_L, V_gt)
        flow_score = (epe - np.min(normal_epe_values)) / (np.max(normal_epe_values) - np.min(normal_epe_values))
        if flow_score > Fl_th:
            print("Anomaly detected at frame", i)
